# Amazon Bedrock AgentCore RuntimeでLangGraphエージェントとAmazon Bedrockモデルをホスティングする

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtimeを使用して既存のエージェントをホストする方法を学びます。

Amazon BedrockモデルとLangGraphの例に焦点を当てます。Amazon BedrockモデルとStrands Agentsについては[こちら](../01-strands-with-bedrock-model)、
OpenAIモデルとStrands Agentsについては[こちら](../03-strands-with-openai-model)をご確認ください。

### チュートリアル詳細

| 項目                | 詳細                                                                               |
|:-------------------|:-----------------------------------------------------------------------------------|
| チュートリアルタイプ      | 対話型                                                                              |
| エージェントタイプ       | 単一                                                                               |
| エージェントフレームワーク | LangGraph                                                                          |
| LLMモデル           | Anthropic Claude Sonnet 3.7                                                       |
| チュートリアル構成要素    | AgentCore Runtimeでのエージェントホスティング。LangGraphとAmazon Bedrockモデルの使用        |
| チュートリアル分野       | 分野横断                                                                            |
| 例の複雑さ           | 簡単                                                                               |
| 使用SDK            | Amazon BedrockAgentCore Python SDKとboto3                                        |

### チュートリアルアーキテクチャ

このチュートリアルでは、既存のエージェントをAgentCore Runtimeにデプロイする方法について説明します。

実演のために、Amazon BedrockモデルとLangGraphエージェントを使用します。

例では、`get_weather`と`get_time`の2つのツールを持つ非常にシンプルなエージェントを使用します。

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### チュートリアルの主要機能

* Amazon Bedrock AgentCore Runtimeでのエージェントホスティング
* Amazon Bedrockモデルの使用
* LangGraphの使用

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* AWS認証情報
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker実行環境

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## エージェントの作成とローカル実験

エージェントをAgentCore Runtimeにデプロイする前に、実験目的でローカルで開発し実行してみましょう。

本格的なエージェントアプリケーションでは、エージェント作成プロセスとエージェント呼び出しプロセスを分離する必要があります。AgentCore Runtimeでは、エージェントの呼び出し部分を`@app.entrypoint`デコレータで装飾し、ランタイムのエントリポイントとします。まず、実験段階での各エージェントの開発方法を見てみましょう。

ここでのアーキテクチャは以下のようになります：

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>

In [ ]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
import argparse
import json
import operator
import math

# Create calculator tool
@tool
def calculator(expression: str) -> str:
    """
    Calculate the result of a mathematical expression.
    
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 3 * 4", "sqrt(16)", "sin(pi/2)")
    
    Returns:
        The result of the calculation as a string
    """
    try:
        # Define safe functions that can be used in expressions
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # Math functions
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # Basic operators (for explicit use)
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # Evaluate the expression safely
        result = eval(expression, safe_dict)
        return str(result)
        
    except ZeroDivisionError:
        return "Error: Division by zero"
    except ValueError as e:
        return f"Error: Invalid value - {str(e)}"
    except SyntaxError:
        return "Error: Invalid mathematical expression"
    except Exception as e:
        return f"Error: {str(e)}"

# Create a custom weather tool
@tool
def weather():
    """Get weather"""  # Dummy implementation
    return "sunny"

# Define the agent using manual LangGraph construction
def create_agent():
    """Create and configure the LangGraph agent"""
    from langchain_aws import ChatBedrock
    
    # Initialize your LLM (adjust model and parameters as needed)
    llm = ChatBedrock(
        model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # or your preferred model
        model_kwargs={"temperature": 0.1}
    )
    
    # Bind tools to the LLM
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # System message
    system_message = "You're a helpful assistant. You can do simple math calculation, and tell the weather."
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        # Add system message if not already present
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

# Initialize the agent
agent = create_agent()

def langgraph_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = langgraph_bedrock(json.loads(args.payload))
    print(response)

#### ローカルエージェントの呼び出し

In [ ]:
!python langgraph_bedrock.py '{"prompt": "What is the weather now?"}'

## AgentCore Runtimeでのデプロイメント用のエージェント準備

それでは、エージェントをAgentCore Runtimeにデプロイしましょう。そのためには以下が必要です：
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp`でRuntime Appをインポート
* `app = BedrockAgentCoreApp()`でコード内にAppを初期化
* 呼び出し関数を`@app.entrypoint`デコレータで装飾
* `app.run()`でAgentCoreRuntimeにエージェントの実行を制御させる

### Amazon BedrockモデルとLangGraph
Amazon Bedrockモデルを使用するLangGraphから始めましょう。他のフレームワークやモデルを使用する例は、親ディレクトリで利用できます。

In [ ]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import argparse
import json
import operator
import math

app = BedrockAgentCoreApp()

# Create calculator tool
@tool
def calculator(expression: str) -> str:
    """
    Calculate the result of a mathematical expression.
    
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 3 * 4", "sqrt(16)", "sin(pi/2)")
    
    Returns:
        The result of the calculation as a string
    """
    try:
        # Define safe functions that can be used in expressions
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # Math functions
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # Basic operators (for explicit use)
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # Evaluate the expression safely
        result = eval(expression, safe_dict)
        return str(result)
        
    except ZeroDivisionError:
        return "Error: Division by zero"
    except ValueError as e:
        return f"Error: Invalid value - {str(e)}"
    except SyntaxError:
        return "Error: Invalid mathematical expression"
    except Exception as e:
        return f"Error: {str(e)}"

# Create a custom weather tool
@tool
def weather():
    """Get weather"""  # Dummy implementation
    return "sunny"

# Define the agent using manual LangGraph construction
def create_agent():
    """Create and configure the LangGraph agent"""
    from langchain_aws import ChatBedrock
    
    # Initialize your LLM (adjust model and parameters as needed)
    llm = ChatBedrock(
        model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # or your preferred model
        model_kwargs={"temperature": 0.1}
    )
    
    # Bind tools to the LLM
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # System message
    system_message = "You're a helpful assistant. You can do simple math calculation, and tell the weather."
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        # Add system message if not already present
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

# Initialize the agent
agent = create_agent()

@app.entrypoint
def langgraph_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    app.run()

## 舞台裏で何が起こっているのか？

`BedrockAgentCoreApp`を使用すると、自動的に以下が行われます：

* ポート8080でリッスンするHTTPサーバーを作成
* エージェントの要件を処理するために必要な`/invocations`エンドポイントを実装
* ヘルスチェック用の`/ping`エンドポイントを実装（非同期エージェントにとって非常に重要）
* 適切なコンテンツタイプとレスポンス形式を処理
* AWS標準に従ったエラーハンドリングを管理

## AgentCore Runtimeへのエージェントデプロイ

`CreateAgentRuntime`操作は包括的な設定オプションをサポートし、コンテナイメージ、環境変数、暗号化設定を指定できます。また、プロトコル設定（HTTP、MCP）と認可メカニズムを設定して、クライアントがエージェントと通信する方法を制御することもできます。

**注意：** 運用のベストプラクティスは、CI/CDパイプラインとIaCを使用してコードをコンテナとしてパッケージ化し、ECRにプッシュすることです。

このチュートリアルでは、Amazon Bedrock AgentCode Python SDKを使用して、アーティファクトを簡単にパッケージ化し、AgentCore Runtimeにデプロイします。

### AgentCore Runtimeデプロイメントの設定

まず、スタータツールキットを使用して、エントリポイント、作成した実行ロール、requirements fileを使ってAgentCore Runtimeデプロイメントを設定します。また、起動時にAmazon ECRリポジトリを自動作成するようにスタータキットを設定します。

設定ステップでは、アプリケーションコードに基づいてDockerファイルが生成されます。

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

agent_name = "langgraph_claude_getting_started"
response = agentcore_runtime.configure(
    entrypoint="langgraph_bedrock.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

### AgentCore Runtimeへのエージェント起動

Dockerファイルが作成されたので、エージェントをAgentCore Runtimeに起動しましょう。これによりAmazon ECRリポジトリとAgentCore Runtimeが作成されます。

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore Runtimeステータスの確認
AgentCore Runtimeをデプロイしたので、そのデプロイメントステータスを確認しましょう。

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtimeの呼び出し

最後に、ペイロードでAgentCore Runtimeを呼び出すことができます。

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "How much is 2+2?"})
invoke_response

### 呼び出し結果の処理

アプリケーションに組み込むために、呼び出し結果を処理することができます。

In [ ]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### boto3でのAgentCore Runtime呼び出し

AgentCore Runtimeが作成されたので、任意のAWS SDKで呼び出すことができます。例えば、boto3の`invoke_agent_runtime`メソッドを使用できます。

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is the weather now?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## クリーンアップ（オプション）

作成されたAgentCore Runtimeをクリーンアップしましょう。

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# おめでとうございます！